In [1]:
import tensorflow as tf
import xgboost as xgb
import pandas as pd
import functools
import time
from tqdm import tqdm
tqdm.pandas()
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score, \
    auc, average_precision_score, pairwise_distances
import scikitplot as skplt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgb_hyper import objective
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pickle
import dill
from functools import partial

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passi

# read gene features

In [73]:
genes = pd.read_csv("data/gene_data/gene_features_all.csv",index_col=0)

# read the training,val,test splits

In [74]:
all_df = pd.read_csv("data/gene_data/allsigs.csv",index_col=0)
val1_df = pd.read_csv("data/gene_data/splits/csv/val_set_1.csv",index_col=0)
val2_df = pd.read_csv("data/gene_data/splits/csv/val_set_2.csv",index_col=0)
val3_df = pd.read_csv("data/gene_data/splits/csv/val_set_3.csv",index_col=0)
val4_df = pd.read_csv("data/gene_data/splits/csv/val_set_4.csv",index_col=0)
test_df = pd.read_csv("data/gene_data/splits/csv/test_set.csv",index_col=0)

valsets = [val1_df,val2_df,val3_df,val4_df]

# only keep labels with more than 3 examples?

In [75]:
all_df = all_df[all_df["moa_count"]>3]
genes = genes.loc[all_df.index]

# turn labels to categorical

In [76]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(all_df.moa_v1)
all_df['moa_categorical'] = le.transform(all_df.moa_v1)

# first remove the test set from all_df and all genes

In [77]:
test_genes = genes.loc[test_df["sig_id"]]
test_sigs = all_df.loc[test_df["sig_id"]]
all_df = all_df.drop(test_df["sig_id"])
genes = genes.drop(test_df["sig_id"])

In [97]:
# defining the space
fspace = {
    'colsample_bylevel' : hp.uniform('colsample_bylevel', 0.1, 1), #+
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.1, 1), #+
    'gamma' : hp.uniform('gamma', 0.1, 1), #+
    'learning_rate' : hp.uniform('learning_rate', 0.1, 1),
    'max_delta_step' : hp.quniform('max_delta_step',1,10,1),
    'max_depth' : hp.quniform('max_depth',6, 12, 1),
    'min_child_weight' : hp.quniform('min_child_weight',10 ,500 ,5),
    'reg_alpha' : hp.uniform('reg_alpha',0.1,100),
    'reg_lambda' : hp.uniform('reg_lambda',0.1,100),
    'subsample' : hp.uniform('subsample',0.1,1.0),
    'max_bin' : hp.quniform('max_bin',16,256,16)
    # add sampling method,max bin,predicto,monotone_constraints,interaction_constraints,single_precision_histogram
}

In [130]:
def objective(fspace, all_df, genes, valsets):
    accs = []
    model_params = {
        "colsample_bylevel" : fspace['colsample_bylevel'],
        "colsample_bytree" : fspace['colsample_bytree'],
        "gamma" : fspace['gamma'],
        "eta" : fspace['learning_rate'],
        "max_delta_step" : int(fspace['max_delta_step']),
        "max_depth" : int(fspace['max_depth']),
        "min_child_weight" : int(fspace['min_child_weight']),
        "alpha" : fspace['reg_alpha'],
        "lambda" : fspace['reg_lambda'],
        "subsample" : fspace['subsample'],
        "objective":'multi:softmax',
        'num_class': all_df['moa_categorical'].nunique(),
        "booster":'gbtree',
        "eval_metric":'merror'
        }
    for i in range(len(valsets)):
        val_genes = genes.loc[valsets[i]["sig_id"]]
        val_sigs = all_df.loc[valsets[i]["sig_id"]]
        train_genes = genes.drop(valsets[i]["sig_id"])
        train_sigs = all_df.drop(valsets[i]["sig_id"])
        dtrain = xgb.DMatrix(data=train_genes, label=train_sigs['moa_categorical'])
        dtest = xgb.DMatrix(data=val_genes, label = val_sigs['moa_categorical'])
        evalist = [(dtest,'eval'),(dtrain,'train')]
        bst = xgb.train(model_params, dtrain, 100, evalist, early_stopping_rounds = 10, verbose_eval = False)
        pred = bst.predict(dtest)
        accs.append(accuracy_score(val_sigs['moa_categorical'], pred))
    ave_acc = np.mean(accs,axis = 0)
    return {'loss': -ave_acc ,  'status': STATUS_OK}

In [131]:
fmin_objective = partial(objective, all_df = all_df, genes = genes, valsets = valsets)

In [136]:
def run_trials():

    trials_step = 1000  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 1  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("my_model.hyperopt", "rb"))
        print("Found saved Trials! Loading...")
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn = fmin_objective, space = fspace, algo=tpe.suggest, max_evals=max_trials, trials=trials)

    print("Best:", best)
    
    # save the trials object
    with open("my_model.hyperopt", "wb") as f:
        pickle.dump(trials, f)
    return(trials)

In [137]:
trials = run_trials()

Found saved Trials! Loading...
Rerunning from 21 trials to 1021 (+1000) trials
100%|█████████████████████████████████████████| 1021/1021 [8:14:38<00:00, 29.07s/trial, best loss: -0.6027980422938406]
Best: {'colsample_bylevel': 0.35788804057451146, 'colsample_bytree': 0.1226530432425497, 'gamma': 0.1571640395272282, 'learning_rate': 0.8098321148828739, 'max_bin': 176.0, 'max_delta_step': 5.0, 'max_depth': 11.0, 'min_child_weight': 10.0, 'reg_alpha': 2.0041642106706092, 'reg_lambda': 69.63017517747672, 'subsample': 0.7429682334802664}


# load the best parameters

In [185]:
best_params = trials.trials[index]['misc']['vals']
hyper_params = {
        "colsample_bylevel" : 0.35788804057451146,
        "colsample_bytree" : 0.1226530432425497,
        "gamma" : 0.1571640395272282,
        "eta" : 0.8098321148828739,
        "max_delta_step" : 5,
        "max_depth" : 11,
        "min_child_weight" : 10,
        "alpha" : 2.0041642106706092,
        "lambda" : 69.63017517747672,
        "subsample" : 0.7429682334802664,
        "objective":'multi:softmax',
        'num_class': all_df['moa_categorical'].nunique(),
        "booster":'gbtree',
        "eval_metric":'merror'
}

In [188]:
accs = []
for i in range(len(valsets)):
    val_genes = genes.loc[valsets[i]["sig_id"]]
    val_sigs = all_df.loc[valsets[i]["sig_id"]]
    train_genes = genes.drop(valsets[i]["sig_id"])
    train_sigs = all_df.drop(valsets[i]["sig_id"])
    dtrain = xgb.DMatrix(data=train_genes, label=train_sigs['moa_categorical'])
    dtest = xgb.DMatrix(data=val_genes, label = val_sigs['moa_categorical'])
    evalist = [(dtest,'eval'),(dtrain,'train')]
    bst = xgb.train(hyper_params, dtrain, 100, evalist, early_stopping_rounds = 10, verbose_eval = False)
    pred = bst.predict(dtest)
    accs.append(accuracy_score(val_sigs['moa_categorical'], pred))
ave_acc = np.mean(accs,axis = 0)

In [189]:
accs

[0.6554621848739496,
 0.5076923076923077,
 0.6326530612244898,
 0.6153846153846154]

# Test performance

In [198]:
val_genes = genes.loc[valsets[0]["sig_id"]]
val_sigs = all_df.loc[valsets[0]["sig_id"]]
train_genes = genes.drop(valsets[0]["sig_id"])
train_sigs = all_df.drop(valsets[0]["sig_id"])
dtrain = xgb.DMatrix(data=train_genes, label=train_sigs['moa_categorical'])
dval = xgb.DMatrix(data=val_genes, label = val_sigs['moa_categorical'])
dtest = xgb.DMatrix(data=test_genes)
evalist = [(dval,'eval'),(dtrain,'train')]
bst = xgb.train(hyper_params, dtrain, 100, evalist, early_stopping_rounds = 10, verbose_eval = False)
pred = bst.predict(dtest)
print(accuracy_score(test_sigs['moa_categorical'], pred))

0.46153846153846156


In [90]:
#le.inverse_transform(pred.astype(int))